# IMPORTS

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import logging
import typing
import gc
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
import torch_xla
from torch import nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig, AutoTokenizer, AutoModelForMaskedLM
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch_xla.core.xla_model as xm


# if you are running this notebook from 'home/dev/enefit/notebook'. 
os.chdir('..') # else adjust to point to the root of the project.

In [3]:
%%capture output

%load_ext kedro.ipython
%reload_kedro

if 'output' in locals() and 'error' in output.stderr:
    output.show()

In [4]:
from src.asrsnlp.pipelines.train_bert import nodes

In [5]:
modelname = catalog.load("params:bert_params.name")
llayer = catalog.load("params:bert_params.llayer")
dropoutratio = catalog.load("params:bert_params.doratio")
max_len = 200
trainparams = catalog.load("params:berttrain_params")

                    INFO     Loading data from 'params:bert_params.name' (MemoryDataset)...     ]8;id=526898;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=527831;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:bert_params.llayer' (MemoryDataset)...   ]8;id=460016;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=823312;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:bert_params.doratio' (MemoryDataset)...  ]8;id=255689;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=824177;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:berttrain_params' (MemoryDataset)...     ]8;id=709889;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=371872;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [6]:
model = nodes.ModelClass(modelname, (768,14), dropoutratio)

In [7]:
tokenizer, model = nodes.import_model(modelname, llayer, dropoutratio)

In [8]:
train_feat = catalog.load("train_cleaned_fea")

[11/29/23 20:52:38] INFO     Loading data from 'train_cleaned_fea' (MlflowParquetDataset)...    ]8;id=734003;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=109990;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [9]:
train_loaded = nodes.loader(train_feat, tokenizer, max_len, trainparams, sample=0.0001)

In [10]:
loss_fn = nodes.get_loss()

In [11]:
optimizer = nodes.get_optimizer(model, 0.01)

In [12]:
device = nodes.get_xla_device()

[11/29/23 20:52:48] WARNING  PJRT is now the default runtime. For more information, see               ]8;id=525479;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py\runtime.py]8;;\:]8;id=82746;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py#35\35]8;;\
                             https://github.com/pytorch/xla/blob/master/docs/pjrt.md                               

                    WARNING  Defaulting to PJRT_DEVICE=CPU                                            ]8;id=825737;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py\runtime.py]8;;\:]8;id=499567;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py#46\46]8;;\

In [13]:
torch.cuda.empty_cache()